In [1]:
%pip install -qU langchain_community beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [5]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

Enter your Groq API key:  ········


In [6]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.atlassian.com/company/careers/details/17350")
page_data = loader.load().pop().page_content
print(page_data)


         Job Details | Atlassian                                  Close   View this page in your language?   All languages   Choose your language    中文   Deutsch   English   Español   Français   Italiano   한국어   Nederlands   日本語   Português   Pусский   Polski                            Get it free     Products    Featured    Developers    Product Managers    IT professionals    Business Teams    Leadership Teams       Featured   Developers   Product Managers   IT professionals   Business Teams   Leadership Teams    See all products       Featured            Jira  Flexible project management             Confluence  Knowledge, all in one place             Jira Service Management  High-velocity service delivery             Trello  Organized & visualized work             Rovo NEW  Unlock enterprise knowledge             Jira Product Discovery NEW  Capture & prioritize ideas             Compass NEW  Optimize software health             Guard NEW  Enhanced cloud security             Loom NE

In [7]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
res

AIMessage(content='[\n  {\n    "role": "Machine learning engineers",\n    "experience": "Bachelor\'s degree or above in Computer Science or a relevant technical field. Currently has or is in the process of obtaining a PhD degree in Computer Science or a relevant technical field.",\n    "skills": [\n      "Strong foundation in AI/ML, LLMs, Optimization and related techniques",\n      "Experience with research, publishing, building and scaling machine learning models using large amounts of data",\n      "Agile development mindset, appreciating the benefit of constant iteration and improvement",\n      "Python programming",\n      "Managing data using Python libraries such as NumPy, Pandas, Matplotlib",\n      "Applied machine learning and deep learning frameworks, like PyTorch",\n      "Natural language processing (NLP), Computer Vision (CV), Reinforcement Learning (RL), and AI Optimization"\n    ],\n    "description": "Join our cutting-edge Machine Learning Research team at Atlassian wh

In [9]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Machine learning engineers',
  'experience': "Bachelor's degree or above in Computer Science or a relevant technical field. Currently has or is in the process of obtaining a PhD degree in Computer Science or a relevant technical field.",
  'skills': ['Strong foundation in AI/ML, LLMs, Optimization and related techniques',
   'Experience with research, publishing, building and scaling machine learning models using large amounts of data',
   'Agile development mindset, appreciating the benefit of constant iteration and improvement',
   'Python programming',
   'Managing data using Python libraries such as NumPy, Pandas, Matplotlib',
   'Applied machine learning and deep learning frameworks, like PyTorch',
   'Natural language processing (NLP), Computer Vision (CV), Reinforcement Learning (RL), and AI Optimization'],
  'description': "Join our cutting-edge Machine Learning Research team at Atlassian where you'll have the opportunity to work on advanced Machine Learning (ML) and

In [10]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [11]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])


In [13]:
links = collection.query(query_texts=['experience in react'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [14]:
job = json_res

In [19]:
job[0]['skills']

['Strong foundation in AI/ML, LLMs, Optimization and related techniques',
 'Experience with research, publishing, building and scaling machine learning models using large amounts of data',
 'Agile development mindset, appreciating the benefit of constant iteration and improvement',
 'Python programming',
 'Managing data using Python libraries such as NumPy, Pandas, Matplotlib',
 'Applied machine learning and deep learning frameworks, like PyTorch',
 'Natural language processing (NLP), Computer Vision (CV), Reinforcement Learning (RL), and AI Optimization']

In [20]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Machine Learning Solutions for Atlassian's Advanced AI Technologies

Dear Hiring Manager,

I hope this email finds you well. I came across the Machine Learning Engineer job posting for your Machine Learning Research team at Atlassian, and I am excited to introduce AtliQ as a potential partner to fulfill your needs.

AtliQ is an AI & Software Consulting company specializing in automating business processes through tailored AI solutions. Our team of experts has empowered numerous enterprises with customized solutions, driving scalability, process optimization, cost reduction, and overall efficiency.

Our capabilities align perfectly with the job description you provided. Our Machine Learning Engineers possess:

1. A strong foundation in AI/ML, LLMs, Optimization, and related techniques
2. Experience in research, publishing, building, and scaling machine learning models using large datasets
3. Proficiency in Python programming, data management using Python libraries (NumPy